# Dependencias

---



In [12]:
# !pip install undetected-chromedriver

# # Primeros pasos con Selenium en Google Colab de la academia https://scraping.link/
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !pip install pandas

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from IPython.display import clear_output 


from datetime import datetime

# from selenium import webdriver
# from selenium.common.exceptions import TimeoutException
# from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from agenda_tools.get_schedule import remove_blank_spaces
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import TimeoutException
from concurrent.futures import ThreadPoolExecutor
from urllib3.exceptions import MaxRetryError

# Funciones

---



In [7]:
def load_next_page(next_page_xpath, links_page_xpath, driver):
  boton_ventana_emergente = None
  box_links = []
  
  try:
    previous_heigth = driver.execute_script('return document.body.scrollHeight')
    while True:
      driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
      time.sleep(2)
      new_heigth = driver.execute_script('return document.body.scrollHeight')
        
      try:
        driver.find_element('xpath', next_page_xpath).click()
        links = [i.get_attribute('href') for i in driver.find_elements('xpath', links_page_xpath)]
        box_links.extend(links)

      except NoSuchElementException:
          print('No hay más Data')
          break
        
      # if new_heigth == previous_heigth:
      #     break
      previous_heigth = new_heigth
    
    try:
      time.sleep(2)
      links = [i.get_attribute('href') for i in driver.find_elements('xpath', links_page_xpath)]
      box_links.extend(links)

    except NoSuchElementException:
        pass

    return list(set(box_links))

  except ElementClickInterceptedException as ex:
      if not boton_ventana_emergente:
          print(f'Hay un objeto bloqueando el boton, vea el error y coloque la direccion del boton que bloquea la pagina en la variable "boton_ventana_emergente". \n-* Error: {ex.msg} -*')
          boton_ventana_emergente = input('Ingrese la ruta Xpath al elemento de la ventana emergente: ') + '[last()]'
      else:
          try:
              driver.find_element('xpath',boton_ventana_emergente).click()
          except Exception:
              print('Error al seleccionar el elemento')

In [8]:
def extract_data(xpath_imagen, xpath_title, xpath_description, driver):
  current_url = driver.current_url
  try:
    imagen = driver.find_element('xpath', xpath_imagen).get_attribute('src')
    if not imagen:
      imagen = driver.find_element('xpath', xpath_imagen).get_attribute('srcset')
    imagen = imagen.split(',')[0] 
  except NoSuchElementException:
    print(f'No se encuentra el elemento imagenes en la pagina {current_url}')
    imagen = None
  try:
    title  = driver.find_element('xpath', xpath_title).text
  except NoSuchElementException:
    print(f'No se encuentra el elementto title en la pagina {current_url}')
    title = None
  try:
    description = driver.find_element('xpath', xpath_description).text
  except NoSuchElementException:
    print(f'No se encuentra el elemento description en la pagina {current_url}')
    description = None
  df = pd.DataFrame([{
      'Title': title,
      'image':imagen,
      'url post': driver.current_url,
      'Description': description,
                 }])
  return df

In [71]:
def go_to_next_page(next_page_xpath, driver):
  boton_ventana_emergente = None
  try:
    next_url = driver.find_element('xpath', next_page_xpath).get_attribute('href')
    print('Next', next_url)
    return next_url, True
  
  except NoSuchElementException:
    next_url= driver.current_url
    return next_url, False

  except ElementClickInterceptedException as ex:
      if not boton_ventana_emergente:
          print(f'Hay un objeto bloqueando el boton, vea el error y coloque la direccion del boton que bloquea la pagina en la variable "boton_ventana_emergente". \n-* Error: {ex.msg} -*')
          boton_ventana_emergente = input('Ingrese la ruta Xpath al elemento de la ventana emergente: ') + '[last()]'
      else:
          try:
              driver.find_element('xpath',boton_ventana_emergente).click()
          except Exception:
              print('Error al seleccionar el elemento')
      

# Start

---



## Iniciar Scrape

In [97]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

# Test

In [ ]:
url_test = input('Ingrese la url de la página con la que quiere hacer la prueba ')
driver.get(url_test)

## Title


In [ ]:
title = '//div[@class="post_info"]/h1'

In [ ]:
try:
  print(driver.find_element('xpath', title).text)
except NoSuchElementException:
  print('no se encuentra el elemento que está buscando')

## Image

In [ ]:
image = '//img[@class="lazyloaded"]'

In [ ]:
try:
  i = driver.find_element('xpath', image).get_attribute('src')
  assert i != None , 'Error'
  print(i)
except AssertionError:
  print(driver.find_element('xpath', image).get_attribute('srcset').split(',')[0])

## Description

In [ ]:
description = ''

In [ ]:
try:
  print(driver.find_element('xpath', description).text)
except NoSuchElementException:
  print('no se encuentra el elemento que está buscando')

# Get Page - Desplegar

In [98]:
# url = input('Inserte la dirección completa de la página web que quiere extraer: ')
url = 'https://www.mdig.com.br/index.php?query=outra&startpos=1'
try:
  driver.get(url)
except MaxRetryError:
  print('Reinicie el Scrape')

# Pagination *Scrape*

In [75]:
next_url

'https://www.mdig.com.br/index.php?query=outra&startpos=11'

In [69]:
driver.find_element('xpath', '//*[contains(text(),"Próximos resultados »")]').get_attribute('href')

'https://www.mdig.com.br/index.php?query=outra&startpos=11'

In [86]:
driver.get('https://www.mdig.com.br/index.php?query=outra&startpos=11')

In [88]:
driver.find_element('xpath', '//*[contains(text(),"Próximos resultados »")]' ).get_attribute('href')

'https://www.mdig.com.br/index.php?query=outra&startpos=21'

In [99]:
next_page_xpath =   '//*[contains(text(),"Próximos resultados »")]'               #input('Ingrese la ruta Xpath al elemento que lleva a la siguiente página: ')
links_page_xpath =  '//h2/a'                                       # Ruta a los links 
xpath_main_images = None

# Adentro de la pagina individual
xpath_imagen = '//img[@class="thbn"]'
xpath_title = '//h1'
xpath_description = '//table[@class="thbody"]'


next_page = True

In [100]:
box = []
box_urls = []
if xpath_main_images:
  images  = []
else:
  images = None
while next_page:
  next_url, next_page = go_to_next_page(next_page_xpath, driver)
  driver.get(next_url)
  box_urls.append(next_url)
  if xpath_main_images:
    images.extend([i.get_attribute('src') for i in driver.find_elements('xpath', xpath_main_images)])
  urls = [i.get_attribute('href') for i in driver.find_elements('xpath', links_page_xpath)]
  for idx, url in enumerate(urls):
    print('Scraping ...')
    try:
        driver.get(url)
        box.append(extract_data(xpath_imagen, xpath_title, xpath_description, driver))
#         if idx % 100 == 0:
#             pass
    except Exception:
        pass
    

    

Next https://www.mdig.com.br/index.php?query=outra&startpos=11
Scraping ...
No se encuentra el elemento imagenes en la pagina https://www.mdig.com.br/index.php?itemid=32186
Scraping ...
Scraping ...
No se encuentra el elemento imagenes en la pagina https://www.mdig.com.br/index.php?itemid=21787
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
No se encuentra el elemento imagenes en la pagina https://www.mdig.com.br/index.php?query=outra&startpos=11#
No se encuentra el elementto title en la pagina https://www.mdig.com.br/index.php?query=outra&startpos=11#
Next https://www.mdig.com.br/index.php?query=outra&startpos=21
Scraping ...
Scraping ...
Scraping ...
No se encuentra el elemento imagenes en la pagina https://www.mdig.com.br/index.php?itemid=15450
No se encuentra el elemento description en la pagina https://www.mdig.com.br/index.php?itemid=15450
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scraping ...
Scrapin

In [101]:
data = box

In [102]:
len(data)

4987

# Load Page Scrape

In [ ]:
next_page_xpath = '//div[@class="search-result-more-txt"]'                         #input('Ingrese la ruta Xpath al elemento que lleva a la siguiente página: ')
links_page_xpath =  '//h3[@class="title"]/a'
xpath_main_images = '//div[@class="thumb"]//img'
#Dentro
xpath_imagen = '//picture/img'
xpath_title = '//h1'
xpath_description = '//p[@class="TextLabel__text-label___3oCVw TextLabel__white___32MyF TextLabel__regular___2X0ym CurrentVideoDetails-current-video-description-1VoK7"]'

In [ ]:
box = []

urls = load_next_page(next_page_xpath, links_page_xpath, driver)
if xpath_main_images:
  images = [i.get_attribute('src') for i in driver.find_elements('xpath', xpath_main_images)]
else:
  images = None
for url in urls:
  driver.get(url)
  box.append(extract_data(xpath_imagen, xpath_title, xpath_description, driver))
    

# Limpieza de datos

---



In [62]:
pd.concat(data).drop_duplicates('Description')

,Title,image,url post,Description
0,Outra jovem teve seu rosto fotochopado para ex...,None,https://www.mdig.com.br/index.php?itemid=32186,"Em junho, a jornalista freelancer Esther Honig..."
0,Mulher tenta em vão estacionar o carro até que...,https://imagens.mdig.com.br/thbs/51578mn.jpg,https://www.mdig.com.br/index.php?itemid=51578,Um vídeo que viralizou nas últimas horas nas r...
0,"Kiira Korp, outra musa da internet (25 fotos)",None,https://www.mdig.com.br/index.php?itemid=21787,"Em junho de 2007, Allison Stokke, uma bela gar..."
0,"Rumsfeld, o dispersivo",https://imagens.mdig.com.br/thbs/3759mn.jpg,https://www.mdig.com.br/index.php?itemid=3759,Ás vezes ficamos p... da vida quando estamos f...
0,Cenas aéreas hipnotizantes de ovelhas sendo pa...,https://imagens.mdig.com.br/thbs/43190mn.jpg,https://www.mdig.com.br/index.php?itemid=43190,"Tim Whitaker, um talentoso cineasta e fotógraf..."
0,Duas convidadas de um casamento brigam após um...,https://imagens.mdig.com.br/thbs/47477mn.jpg,https://www.mdig.com.br/index.php?itemid=47477,"Na Indonésia, duas convidadas de um casamento ..."
0,"Sol emite outra grande erupção, mas sem conseq...",https://imagens.mdig.com.br/thbs/14091mn.jpg,https://www.mdig.com.br/index.php?itemid=14091,Nomeada como a mancha solar 1105 pelo Solar Dy...
0,"Em um jogo de beisebol, mulher pega a bolinha ...",https://imagens.mdig.com.br/thbs/52011mn.jpg,https://www.mdig.com.br/index.php?itemid=52011,"Nas redes sociais, se tornaram virais as image..."
0,Gaivota folgada pega carona no lombo de outra ...,https://imagens.mdig.com.br/thbs/51683mn.jpg,https://www.mdig.com.br/index.php?itemid=51683,"Já vimos coisas estranhas em nosso tempo, mas ..."
0,Motorista do Uber persegue outra motorista que...,https://imagens.mdig.com.br/thbs/47098mn.jpg,https://www.mdig.com.br/index.php?itemid=47098,O incidente aconteceu no início do ano em Fort...


In [103]:
df = pd.concat(box)
if images:
  df['Main_images'] = images
# df.drop_duplicates(inplace=True)
df.dropna(subset=['Title'], inplace=True)

In [104]:
columns = df.columns
df.rename(columns={columns[1]:'Description', columns[2]:'Description'},inplace= True)

In [105]:
df.head()

,Title,Description,Description,Description
0,Outra jovem teve seu rosto fotochopado para ex...,None,https://www.mdig.com.br/index.php?itemid=32186,"Em junho, a jornalista freelancer Esther Honig..."
0,Mulher tenta em vão estacionar o carro até que...,https://imagens.mdig.com.br/thbs/51578mn.jpg,https://www.mdig.com.br/index.php?itemid=51578,Um vídeo que viralizou nas últimas horas nas r...
0,"Kiira Korp, outra musa da internet (25 fotos)",None,https://www.mdig.com.br/index.php?itemid=21787,"Em junho de 2007, Allison Stokke, uma bela gar..."
0,"Rumsfeld, o dispersivo",https://imagens.mdig.com.br/thbs/3759mn.jpg,https://www.mdig.com.br/index.php?itemid=3759,Ás vezes ficamos p... da vida quando estamos f...
0,Cenas aéreas hipnotizantes de ovelhas sendo pa...,https://imagens.mdig.com.br/thbs/43190mn.jpg,https://www.mdig.com.br/index.php?itemid=43190,"Tim Whitaker, um talentoso cineasta e fotógraf..."


In [106]:
name = 'results_' + datetime.now().strftime('%d_%m_%y_%H_%M') + '.xlsx'

In [107]:
df.to_excel( name, index=False)

In [108]:
df.shape

(4533, 4)

In [ ]:
df.to_excel('/content/gdrive/MyDrive/'+ name, index=False)